In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/DataSet1.zip', 'r') as zip_ref:
        zip_ref.extractall('./DataSet1')
except:
    print("Using Local Machine")
!git clone https: // github.com/ultralytics/yolov5.git
!pip install - r yolov5/requirements.txt


In [ ]:
# Include all packages
import os
import gc
import cv2
import shutil
import random
import numpy as np
import pandas as pd
from time import time
from copy import deepcopy

from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from yolov5.models.yolo import Model
from sklearn.model_selection import train_test_split

import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [ ]:
def CannyEdge(capturedImage):
    grayScale = cv2.cvtColor(capturedImage, cv2.COLOR_BGR2GRAY)
    constrastKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    topHat = cv2.morphologyEx(grayScale, cv2.MORPH_TOPHAT, constrastKernel)
    blackHat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, constrastKernel)
    grayScale = grayScale + topHat - blackHat
    gaussianImage = cv2.GaussianBlur(grayScale, (3, 3), 0)
    imageMedian = np.median(capturedImage)
    lowerThreshold = max(0, (0.7 * imageMedian))
    upperThreshold = min(255, (0.7 * imageMedian))
    cannyEdgeImage = cv2.Canny(gaussianImage, lowerThreshold, upperThreshold)
    cannyEdgeImage = cv2.bitwise_not(cannyEdgeImage)
    cannyEdgeImage = cv2.cvtColor(cannyEdgeImage, cv2.COLOR_GRAY2BGR)
    return cannyEdgeImage


In [ ]:
def ResizeImage(image: np.ndarray, x1: int, y1: int, x2: int, y2: int, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    widthScale = newWidth / originalWidth
    heightScale = newHeight / originalHeight
    x1New, y1New = int(x1 * widthScale), int(y1 * heightScale)
    x2New, y2New = int(x2 * widthScale), int(y2 * heightScale)
    return resizedImage, x1New, y1New, x2New, y2New


In [ ]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    resize = (640, 640)
    annotationsFilePath = dataSetFolderPath+"/annotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=",")
    uniqueSigns = annotationsDataFrame['class'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        [classIndex, x1, y1, x2, y2] = [uniqueSigns.index(row[5]), row[1], row[2], row[3], row[4]]
        resizedImage, x1New, y1New, x2New, y2New = ResizeImage(image, x1, y1, x2, y2, resize[0], resize[1])
        # resizedImage = CannyEdge(resizedImage)
        images.append(resizedImage)
        annotations.append(
            [classIndex, x1New, y1New, x2New, y2New])
    del annotationsDataFrame

    X_train, X_val, y_train, y_val = train_test_split(images, annotations, test_size=0.2, random_state=42)

    return len(uniqueSigns), X_train, X_val, y_train, y_val


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label).float()
        return inputData, label


def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [ ]:
def CreateYolov5Model(numClasses: int, version: str, device):
    congfigFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(congfigFile, ch=3, nc=numClasses)
    return model

def LoadModel(yolov5Model):
    savedModels = [ modelName for modelName in os.listdir('/content/drive/MyDrive/DL Project/Trained Models/') if ('yolov5Modelv2' in modelName)]
    savedModels.sort()
    print(savedModels[-1])
    yolov5Model.load_state_dict(torch.load('/content/drive/MyDrive/DL Project/Trained Models/'+savedModels[-1], map_location=torch.device(device)))
    return yolov5Model

In [ ]:
def DetectImage(model, inputs, device, conf_thres=0.2, iou_thres=0.5):
    model.eval()

    inputs = torch.tensor(inputs, dtype=torch.float32)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.permute(0, 3, 1, 2)
    inputs = inputs.to(device)
    conf_thres = torch.tensor(conf_thres)
    with torch.no_grad():
        output = model(inputs)
        output = output[0]
        confidences = output[..., 4:5]
        max_confidences, max_indices = torch.max(confidences, dim=1)
        box_coordinates = output[..., :4].view(-1, 4)
        confidence_scores = output[..., 4].view(-1)
        nms_indices = torchvision.ops.nms(box_coordinates, confidence_scores, iou_thres)
        # output = output.view(-1, output.shape[-1])[nms_indices]
        output = output.view(-1, output.shape[-1])[max_indices]
    output = output.squeeze(0)
    return output


In [ ]:
batchSize = 32
inputShape = (640, 640)
epochs = 300
numAnchors = 3
yolo5Version = 'm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
print("Using {} device".format(device))


In [ ]:
numClasses, X_train, X_val, y_train, y_val = LoadDataSet("./DataSet1")
print(numClasses)
gc.collect()


In [ ]:
# try:
#     from google.colab.patches import cv2_imshow
#     cv2_imshow(X_train[10])
# except:
#     print("using Local")
#     cv2.imshow("Input Image", X_train[10])

In [ ]:
yolov5Model = CreateYolov5Model(numClasses, yolo5Version, device)
yolov5Model = LoadModel(yolov5Model)
yolov5Model = yolov5Model.to(device)

In [ ]:
randInt = random.randint(0,len(X_val))
image = X_val[randInt]
image1 = deepcopy(image)
predictions = DetectImage(yolov5Model, image, device)
[a1,b1,a2,b2] = y_val[randInt]
bBoxs = []
machingbBoxes = []
albBoxs =[]

for pred in predictions:
    x1, y1, x2, y2, m1,m2 = pred[:6]
    m1,m2, x1, y1, x2, y2= int(m1), int(m2),int(x1), int(y1), int(x2), int(y2)
    if(a1 == x1 or a2 == x2 or b1 == y1 or b2 == y2 ):
      if(((x1-x2) >= 17 and (x1-x2) <= 32) and ((y1-y2) >= 31 and (y1-y2)<= 56) ):
        machingbBoxes.append([x1, y1, x2,y2])
      
    if(abs(x1-x2) >= 17  and abs(y1-y2) >= 31 ):
        albBoxs.append([x1, y1, x2, y2])

    if((abs(x1-x2) >= 17 and abs(x1-x2) <= 32) and (abs(y1-y2) >= 31 and abs(y1-y2)<= 56) ):
        bBoxs.append([x1, y1, x2, y2])
    
    
    # x_center, y_center, width, height =x1, y1, x2, y2
    # x1 = x_center - (width // 2)
    # y1 = y_center - (height // 2)
    # x2 = x_center + (width // 2)
    # y2 = y_center + (height // 2)

print("No. machingbBoxes detected:" ,len(machingbBoxes) )
print("No. albBoxs detected:" ,len(albBoxs) )
print("No. bBoxs detected:" ,len(bBoxs) )




for bBox in machingbBoxes:
    [x1, y1, x2, y2] = bBox
    cv2.rectangle(image, (x1, y1), (x2, y2), (0,0,0), 2)

for bBox in albBoxs:
    [x1, y1, x2, y2] = bBox
    cv2.rectangle(image, (x1, y1), (x2, y2), (0,0,255), 2)

for bBox in bBoxs:
    [x1, y1, x2, y2] = bBox
    cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)

cv2.rectangle(image, (a1, b1), (a2, b2), (0,255,0), 2)


try:
    from google.colab.patches import cv2_imshow
    cv2_imshow(image)
except:
    print("using Local")
    cv2.imshow("Input Image", image)



